In [54]:
import pandas as pd
from time import time
import numpy as np

In [28]:
pkl=pd.read_pickle(r'\\fatherserverdw\Q\research\images\skin_aging\wsi\hovernet_out\df\12.pkl')

In [30]:
start = time()
pkls =pd.read_feather(r'\\fatherserverdw\q\research\images\skin_aging\wsi\hovernet_out\df\2d_skin_hovernet.ftr') #42sec local # 90sec network
print(round(time()-start),'seconds')

125 seconds


In [32]:
pkls=pkls[pkls['inroi']>0].reset_index(drop=True)
pkls.to_feather(r'\\fatherserverdw\q\research\images\skin_aging\wsi\hovernet_out\df\2d_skin_hovernet_inroi.ftr')

In [33]:
pkls['inroi'].unique()

array([1, 3, 4, 2, 5, 6], dtype=int64)

In [3]:
agelut = pd.read_excel(r'\\fatherserverdw\Q\research\source code\skin_aging_matlab\skin_samples.xlsx', sheet_name='imlist')

In [8]:
len(pkls)

3101441

In [14]:

def redcap2age(redcap):
    return agelut['Age'][agelut['redcap']==redcap]


In [55]:
%%time
avgs = []
for imid in pkls['imID'].unique():
    pkl = pkls[pkls['imID']==imid]
    pkl = pkl.drop(columns=['bbox', 'centroid','contour'])
    for secid in pkl['inroi'].unique():
        pk = pkl[pkl['inroi']==secid]
        for type in range(1,13):
            p = pk[pk['type']==type]
            pkmean = p.mean(axis=0)
            pkmean['imID'] = imid
            pkstd = p.std(axis=0)
            pkstd['imID'] = imid
            pk = pd.concat([pkmean,pkstd])
            pk['Age'] = agelut['Age'][agelut['redcap']==imid].values[0]
            avgs.append(pkmean.replace(np.nan, 0))
avgdf = pd.DataFrame(avgs)

CPU times: total: 12.3 s
Wall time: 8.3 s


In [60]:
pk = pd.concat([pkmean,pkmean])


In [56]:
avgdf.to_csv('avgdf.csv')

In [ ]:
avg_lvl_corr=[]
for type in avgdf['type'].unique():
    adf = avgdf[avgdf['type']==type]
    corr = adf.corrwith(adf['age'], method='kendall')
    avg_lvl_corr.append(corr)